In [1]:
import cv2
from ultralytics import YOLO
import numpy as np

model = YOLO(r'D:\OneDrive\Bangkit Academy\Project\AR\Test\new_ar.tflite') 

cap = cv2.VideoCapture(0)
baju = cv2.imread('test3.png', cv2.IMREAD_UNCHANGED)

if not cap.isOpened():
    print("Error: Camera is not available.")
    exit()

while True:
    
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    results = model(frame)
    annotated_frame = results[0].plot()
    boxes = results[0].boxes.xyxy.tolist()

    if len(boxes) > 0 and baju.shape[2] == 4:  
        x1, y1, x2, y2 = map(int, boxes[0])  
        original_width = x2 - x1
        width = original_width + 80  # Tambahkan 100 piksel untuk lebar
        height = (y2 - y1) * 7  # Tinggi yang diperbesar
        resized_baju = cv2.resize(baju, (width, height))

        # Sesuaikan x1 agar citra diperluas simetris ke kiri dan kanan, dari 100 dibagi 2
        x1_adjusted = max(x1 - 40, 0)
        
        canvas_height = max(frame.shape[0], y1 + height)
        canvas_width = max(frame.shape[1], x1_adjusted + width)
        canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
        canvas[:frame.shape[0], :frame.shape[1], :] = frame

        mask = resized_baju[:, :, 3]
        baju_rgb = resized_baju[:, :, :3]
        
        canvas_roi = canvas[y1:y1+height, x1_adjusted:x1_adjusted+width]
        mask_inv = cv2.bitwise_not(mask)
        roi_bg = cv2.bitwise_and(canvas_roi, canvas_roi, mask=mask_inv)
        baju_fg = cv2.bitwise_and(baju_rgb, baju_rgb, mask=mask)
        canvas[y1:y1+height, x1_adjusted:x1_adjusted+width] = cv2.add(roi_bg, baju_fg)
        
    
        frame = canvas
    cv2.imshow('YOLO Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading D:\OneDrive\Bangkit Academy\Project\AR\Test\new_ar.tflite for TensorFlow Lite inference...

0: 640x640 (no detections), 242.6ms
Speed: 11.0ms preprocess, 242.6ms inference, 352.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 236.8ms
Speed: 5.0ms preprocess, 236.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 216.0ms
Speed: 5.0ms preprocess, 216.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 215.1ms
Speed: 4.0ms preprocess, 215.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 214.2ms
Speed: 5.0ms preprocess, 214.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 222.2ms
Speed: 4.0ms 